In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.utils import class_weight
import random
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Input
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import EfficientNetB4
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
import tensorflow_addons as tfa
import albumentations as A
from functools import partial
import gc

import efficientnet.tfkeras as efn
#from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
mixed_precision.set_policy(policy)

AUTOTUNE = tf.data.experimental.AUTOTUNE

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla V100-SXM2-16GB, compute capability 7.0
Instructions for updating:
Use tf.keras.mixed_precision.LossScaleOptimizer instead. LossScaleOptimizer now has all the functionality of DynamicLossScale


In [7]:
import random
from numpy.random import seed
from tensorflow.random import set_seed

seed_value = 42
random.seed(seed_value)
seed(seed_value)
set_seed(seed_value)

In [8]:
!ls

 Cassava_Production_Notebook.ipynb	  'effnetb4 3fold.h5'
 cassava-leaf-disease-classification.zip  'effnetb4 4fold.h5'
 data					  'effnetb4 5fold.h5'
'denoised effnetb4 1fold.h5'		   nohup.out
'effnetb0 1fold.h5'			   pytorch-and-tf-with-jupyter.def
'effnetb0 2fold.h5'			   pytorch_21.03-py3.sif
'effnetb0 3fold.h5'			   tensorflow-with-jupyter.def
'effnetb0 4fold.h5'			   tensorflow-with-jupyter.sif
'effnetb0 5fold.h5'			   tensorflow_21.02-tf2-py3.sif
'effnetb4 1fold.h5'			   tensorflow_21.03-tf2-py3.sif
'effnetb4 2fold.h5'


In [9]:
!ls -la data

total 5409
drwxr-xr-x  6 tug98850 cis      9 Mar 23 19:36 .
drwx------ 14 tug98850 cis     40 Apr  2 00:02 ..
-rw-r--r--  1 tug98850 cis    172 Nov 25 16:35 label_num_to_disease_map.json
-rw-r--r--  1 tug98850 cis     32 Nov 25 16:35 sample_submission.csv
drwxr-xr-x  2 tug98850 cis      3 Mar 23 19:33 test_images
drwxr-xr-x  2 tug98850 cis      3 Mar 23 19:33 test_tfrecords
-rw-r--r--  1 tug98850 cis 358283 Nov 25 16:35 train.csv
drwxr-xr-x  2 tug98850 cis  21399 Mar 23 19:33 train_images
drwxr-xr-x  2 tug98850 cis     18 Mar 23 19:34 train_tfrecords


In [10]:
df_train = pd.read_csv("./data/train.csv")
training_folder = './data/train_images/'
df_train["filepath"] = training_folder + df_train["image_id"]
df_train.head()

,image_id,label,filepath
0,1000015157.jpg,0,./data/train_images/1000015157.jpg
1,1000201771.jpg,3,./data/train_images/1000201771.jpg
2,100042118.jpg,1,./data/train_images/100042118.jpg
3,1000723321.jpg,1,./data/train_images/1000723321.jpg
4,1000812911.jpg,3,./data/train_images/1000812911.jpg


In [11]:
batch_size = 16
# original image_size = 224
image_size = 380
input_shape = (image_size, image_size, 3)
epochs = 15
CLASSES = ['0', '1', '2', '3', '4']

In [12]:
skf = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed_value)
for train_index, val_index in skf.split(df_train["image_id"], df_train["label"]):
    train_data = df_train.loc[train_index]
    val_data = df_train.loc[val_index]

In [13]:
def load_image_and_label_from_path(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img, label

In [14]:
training_data = tf.data.Dataset.from_tensor_slices((train_data["filepath"].values, train_data["label"].values))
validation_data = tf.data.Dataset.from_tensor_slices((val_data["filepath"].values, val_data["label"].values))

training_data = training_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
validation_data = validation_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)


print("number of training images: {ti} \t number of validation images: {vi}".format(ti=len(training_data), vi=len(validation_data)))

number of training images: 17117 	 number of validation images: 4280


## Image Augmentation
Map the image augmentations to the tf tensors

In [15]:
def augment_train_data(train_ds):
    transforms = A.Compose([
            A.RandomResizedCrop(image_size, image_size),
            A.Transpose(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            A.CoarseDropout(p=0.5),
            A.Cutout(p=0.5),
            ], p=1)
    
    def aug_fn(image):
        data = {"image":image}
        aug_data = transforms(**data)
        aug_img = aug_data["image"]
        aug_img = tf.cast(aug_img, tf.float32)
        return aug_img

    def process_data(image, label):
        aug_img = tf.numpy_function(func=aug_fn, inp=[image], Tout=tf.float32)
        return aug_img, label
    
    def set_shapes(img, label, img_shape=(image_size,image_size,3)):
        img.set_shape(img_shape)
        label.set_shape([])
        return img, label
    
    ds_alb = train_ds.map(partial(process_data), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
    ds_alb = ds_alb.map(set_shapes, num_parallel_calls=AUTOTUNE)
    ds_alb = ds_alb.repeat()
    ds_alb = ds_alb.batch(batch_size)
    return ds_alb

In [16]:
def augment_val_data(val_ds):
    transforms = A.Compose([
                A.CenterCrop(image_size, image_size),
                ], p=1)
    
    def aug_fn(image):
        data = {"image":image}
        aug_data = transforms(**data)
        aug_img = aug_data["image"]
        aug_img = tf.cast(aug_img, tf.float32)
        return aug_img

    def process_data(image, label):
        aug_img = tf.numpy_function(func=aug_fn, inp=[image], Tout=tf.float32)
        return aug_img, label
    
    def set_shapes(img, label, img_shape=(image_size, image_size,3)):
        img.set_shape(img_shape)
        label.set_shape([])
        return img, label
    
    ds_alb = val_ds.map(partial(process_data), num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)
    ds_alb = ds_alb.map(set_shapes, num_parallel_calls=AUTOTUNE).batch(batch_size)
    return ds_alb

### The Model and Training
Create and run the model

In [17]:
def create_model():
    #Creating the model
  
    #preproccess to resize to 224, because it's what B0 calls for
    #img_adjust_layer = tf.keras.layers.Lambda(tf.keras.applications.efficientnet.preprocess_input, input_shape=input_shape)

    # customized model
    #base_model = efn.EfficientNetB6(weights='noisy-student',
                                    #include_top=False)
    

    base_model = efn.EfficientNetB4(weights='noisy-student',
                                    include_top=False,
                                    input_shape=input_shape)
    
    #rebuild top
    inputs = Input(shape=input_shape)
    built_model = base_model(inputs)
    pooling = GlobalAveragePooling2D()(built_model)
    dropout=Dropout(0.3)(pooling)
    dense=Dense(8, activation='relu')(dropout)
    outputs=Dense(5, activation='softmax', dtype='float32')(dense)

    #compile
    model = Model(inputs=inputs, outputs=outputs)
    
    #learning rate scheduler
    # adjust this based on steps
    lr_scheduler = keras.experimental.CosineDecay(
        initial_learning_rate=1e-3,
        decay_steps=(int(len(df_train)*0.04/batch_size)+1)*epochs)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler, epsilon=0.001),
        loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01),  
        metrics=['accuracy'])
    
    return model

In [18]:
def train_val_split(train_data, val_data):
    training_data = tf.data.Dataset.from_tensor_slices((train_data["filepath"].values, train_data["label"].values))
    validation_data = tf.data.Dataset.from_tensor_slices((val_data["filepath"].values, val_data["label"].values))

    training_data = training_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
    validation_data = validation_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE)
    
    return (training_data, validation_data)

In [19]:
epochs = 20
# try adjusting this
total_steps = (int(len(df_train)*0.08/batch_size)+1)
#total_steps = len(training_data) // batch_size
#DON'T FORGET TO RESET FOLD NUMBER TO 0
fold_number = 0
n_splits = 5
train_list = []
val_list = []

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed_value)
for train_index, val_index in skf.split(df_train["image_id"], df_train["label"]):
    train_list.append(train_index)
    val_list.append(val_index)

In [20]:
# for optimization
def onehot(image,label):
    CLASSES = 5
    return image,tf.one_hot(label,CLASSES)

## The Model and Training
Create and run the model

In [21]:
oof_accuracy = []

for i in range(n_splits-fold_number): 
    # clearing memory
    tf.keras.backend.clear_session()
    gc.collect()
    
    # data split
    train_set = df_train.loc[train_list[fold_number]]
    val_set = df_train.loc[val_list[fold_number]]
    
    train_data, val_data_ = train_val_split(train_set, val_set)
    
    train_alb = augment_train_data(train_data)
    val_alb = augment_val_data(val_data_)
    
    #train_final, val_final = optimize_data(train_alb, val_alb)
    
    train_alb_mod = train_alb.map(onehot, num_parallel_calls=AUTOTUNE)
    train_final = train_alb_mod.prefetch(buffer_size=AUTOTUNE)
    
    val_alb_mod = val_alb.map(onehot, num_parallel_calls=AUTOTUNE)
    val_final = val_alb_mod.prefetch(buffer_size=AUTOTUNE)
    
    # create the model
    model = create_model()
    
    print("Training fold no.: " + str(fold_number+1))

    model_name = "effnetb4 "
    fold_name = "fold.h5"
    filepath = model_name + str(fold_number+1) + fold_name
    callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
                 ModelCheckpoint(filepath=filepath, monitor='val_accuracy', save_best_only=True)]

    # train
    history = model.fit(train_final, steps_per_epoch=total_steps, epochs=epochs, validation_data=val_final, callbacks=callbacks)
    oof_accuracy.append(max(history.history["val_accuracy"]))
    fold_number += 1
    if fold_number == n_splits:
        print("Training finished!")

  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)

Training fold no.: 1
Epoch 1/20
107/107 [==============================] - 138s 586ms/step - loss: 1.3024 - accuracy: 0.5216 - val_loss: 1.0934 - val_accuracy: 0.5771
Epoch 2/20
107/107 [==============================] - 42s 391ms/step - loss: 0.8500 - accuracy: 0.6769 - val_loss: 0.6824 - val_accuracy: 0.7582
Epoch 3/20
107/107 [==============================] - 43s 407ms/step - loss: 0.8064 - accuracy: 0.7344 - val_loss: 0.7630 - val_accuracy: 0.7556
Epoch 4/20
107/107 [==============================] - 43s 402ms/step - loss: 0.7065 - accuracy: 0.7603 - val_loss: 0.6290 - val_accuracy: 0.8023
Epoch 5/20
107/107 [==============================] - 42s 397ms/step - loss: 0.6790 - accuracy: 0.7768 - val_loss: 0.6537 - val_accuracy: 0.7876
Epoch 6/20
107/107 [==============================] - 41s 386ms/step - loss: 0.5869 - accuracy: 0.8064 - val_loss: 0.5426 - val_accuracy: 0.8264
Epoch 7/20
107/107 [=================

107/107 [==============================] - 44s 410ms/step - loss: 0.6073 - accuracy: 0.8031 - val_loss: 0.5240 - val_accuracy: 0.8465
Epoch 14/20
107/107 [==============================] - 42s 397ms/step - loss: 0.5989 - accuracy: 0.8163 - val_loss: 0.5265 - val_accuracy: 0.8441
Epoch 15/20
107/107 [==============================] - 43s 402ms/step - loss: 0.5855 - accuracy: 0.8157 - val_loss: 0.5260 - val_accuracy: 0.8448
Epoch 16/20
107/107 [==============================] - 42s 395ms/step - loss: 0.6061 - accuracy: 0.7968 - val_loss: 0.5271 - val_accuracy: 0.8458
Epoch 17/20
107/107 [==============================] - 42s 399ms/step - loss: 0.6006 - accuracy: 0.8089 - val_loss: 0.5246 - val_accuracy: 0.8467
Epoch 18/20
107/107 [==============================] - 43s 400ms/step - loss: 0.5857 - accuracy: 0.8030 - val_loss: 0.5262 - val_accuracy: 0.8462
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
  opt = tf.keras.mixed_precision.experimental.LossSc

In [22]:
print("Average Out-Of-Fold Accuracy: {:.2f}".format(np.mean(oof_accuracy)))

Average Out-Of-Fold Accuracy: 0.86


In [23]:
# First we load our models
models = [] 
for i in range(5):
    effnet = load_model("./effnetb4 " + str(i+1) + "fold.h5")
    models.append(effnet)

model_one = models[0]
model_two = models[1]
model_three = models[2]
model_four = models[3]
model_five = models[4]

In [24]:
# Then we get our validation data
df = pd.read_csv("./data/train.csv")
val_list = []

skf = StratifiedKFold(n_splits=5)
for train_index, val_index in skf.split(df["image_id"], df["label"]):
    val_list.append(val_index)

one_fold = df.loc[val_list[0]]
two_fold = df.loc[val_list[1]]
three_fold = df.loc[val_list[2]]
four_fold = df.loc[val_list[3]]
five_fold = df.loc[val_list[4]]

## Test Time Augmentation

In [25]:
tta = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.25),
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.2)
    ]
)

In [26]:
def duplicate_image(img_path, image_size=image_size, tta_runs=2):

    img = Image.open(img_path)
    img = img.resize((image_size, image_size))
    img_height, img_width = img.size
    img = np.array(img)
    
    img_list = []
    for i in range(tta_runs):
        img_list.append(img)
  
    return np.array(img_list)

In [27]:
def predict_with_tta(image_filename, folder, tta_runs=2):
    
    #apply TTA to each of the 3 images and sum all predictions for each local image
    localised_predictions = []
    local_image_list = duplicate_image(folder+image_filename)
    for local_image in local_image_list:
        local_image = tf.expand_dims(local_image,0)
        augmented_images = [tta(local_image) for i in range(tta_runs)]
        predictions = model.predict(np.array(augmented_images[0]))
        localised_predictions.append(np.sum(predictions, axis=0))
    
    #sum all predictions from all 3 images and retrieve the index of the highest value
    global_predictions = np.sum(np.array(localised_predictions),axis=0)
    max_value = max(global_predictions)
    final_prediction = np.argmax(global_predictions)
    
    return [final_prediction, max_value, global_predictions]

In [28]:
train_folder = "./data/train_images/"
train_image = "1000015157.jpg"
predictions = predict_with_tta(train_image, train_folder)

print("Predicted Label: ", predictions[0])
print("Predicted Label Value: ", predictions[1])
print("Predicted One-Hot Label: ", predictions[2])

Predicted Label:  4
Predicted Label Value:  0.7359253
Predicted One-Hot Label:  [0.6148509  0.49956995 0.11468517 0.03496872 0.7359253 ]


In [29]:
print("Confidence Level: {:.2f}".format(predictions[1]/2*100), "%")

Confidence Level: 36.80 %


## Get Out-Of-Fold Predictions

In [30]:
def predict_image_list(image_list, folder):
    predictions = []
    values = []
    with tqdm(total=len(image_list)) as pbar:
        for image_filename in image_list:
            pbar.update(1)
            predictions.append(predict_with_tta(image_filename, folder)[0])
            values.append(predict_with_tta(image_filename, folder)[1])
    return [predictions, values]

In [31]:
model = model_one
placeholder = predict_image_list(one_fold["image_id"], train_folder)
one_fold["pred"] = placeholder[0]
one_fold["value"] = placeholder[1]
one_fold.head()

100%|██████████| 4280/4280 [36:37<00:00,  1.95it/s]


,image_id,label,pred,value
0,1000015157.jpg,0,4,1.191875
1,1000201771.jpg,3,3,1.900006
2,100042118.jpg,1,4,1.722907
3,1000723321.jpg,1,1,1.088217
4,1000812911.jpg,3,3,1.959418


In [32]:
model = model_two
placeholder = predict_image_list(two_fold["image_id"], train_folder)
two_fold["pred"] = placeholder[0]
two_fold["value"] = placeholder[1]
two_fold.head()

100%|██████████| 4280/4280 [36:59<00:00,  1.93it/s]


,image_id,label,pred,value
4152,1741376467.jpg,2,2,1.718303
4160,1742921296.jpg,2,3,1.378709
4165,1744132907.jpg,2,2,1.736743
4170,1745213235.jpg,2,2,1.892229
4186,1748078047.jpg,2,2,0.863447


In [33]:
model = model_three
placeholder = predict_image_list(three_fold["image_id"], train_folder)
three_fold["pred"] = placeholder[0]
three_fold["value"] = placeholder[1]
three_fold.head()

100%|██████████| 4279/4279 [38:11<00:00,  1.87it/s]


,image_id,label,pred,value
8381,248855703.jpg,2,2,1.916682
8384,2488945331.jpg,2,2,1.969591
8403,2491752039.jpg,2,2,1.657983
8407,2492550594.jpg,2,2,1.334445
8416,2494764703.jpg,2,2,1.972245


In [34]:
model = model_four
placeholder = predict_image_list(four_fold["image_id"], train_folder)
four_fold["pred"] = placeholder[0]
four_fold["value"] = placeholder[1]
four_fold.head()

100%|██████████| 4279/4279 [38:13<00:00,  1.87it/s]


,image_id,label,pred,value
12789,3290333742.jpg,2,2,1.248505
12795,3292343702.jpg,2,3,1.777464
12801,3293012629.jpg,2,2,1.916770
12810,3294433487.jpg,1,1,1.116190
12813,329471118.jpg,1,1,1.937912


In [35]:
model = model_five
placeholder = predict_image_list(five_fold["image_id"], train_folder)
five_fold["pred"] = placeholder[0]
five_fold["value"] = placeholder[1]
five_fold.head()

100%|██████████| 4279/4279 [37:18<00:00,  1.91it/s] 


,image_id,label,pred,value
16944,4048377608.jpg,2,2,1.970032
16951,4049425598.jpg,2,3,1.849593
16954,4049743612.jpg,2,2,1.928237
16956,4050218395.jpg,2,2,1.957494
16958,4050718274.jpg,2,4,1.363522


In [36]:
threshold = 2*0.8

In [37]:
mask1 = (one_fold["label"] != one_fold["pred"]) & (one_fold["value"] >= threshold)
one_list = one_fold[mask1].index.to_list()

mask2 = (two_fold["label"] != two_fold["pred"]) & (two_fold["value"] >= threshold)
two_list = two_fold[mask2].index.to_list()

mask3 = (three_fold["label"] != three_fold["pred"]) & (three_fold["value"] >= threshold)
three_list = three_fold[mask3].index.to_list()

mask4 = (four_fold["label"] != four_fold["pred"]) & (four_fold["value"] >= threshold)
four_list = four_fold[mask4].index.to_list()

mask5 = (five_fold["label"] != five_fold["pred"]) & (five_fold["value"] >= threshold)
five_list = five_fold[mask5].index.to_list()

combined_list = list(np.unique(one_list + two_list + three_list + four_list + five_list))

In [38]:
temp = df_train.iloc[combined_list]
temp["label"].value_counts()

2    183
4    117
1     61
3     58
0     52
Name: label, dtype: int64

In [39]:
pct = len(temp)/len(df_train)*100
print("Percentage of Data To Be Removed: {:.2f}".format(pct), "%")

Percentage of Data To Be Removed: 2.20 %


## Drop these images & Re-fit the Model

In [40]:
df = df_train.drop(combined_list, axis="index")

In [41]:
df.reset_index(drop=True, inplace=True)

In [42]:
fold_number = 0
n_splits = 5
oof_accuracy = []

tf.keras.backend.clear_session()
skf = StratifiedKFold(n_splits=n_splits)
for train_index, val_index in skf.split(df["image_id"], df["label"]):
    # clearing memory
    tf.keras.backend.clear_session()
    gc.collect()
    
    # data split
    train_set = df.loc[train_index]
    val_set = df.loc[val_index]
    
    train_data, val_data_ = train_val_split(train_set, val_set)
    
    train_alb = augment_train_data(train_data)
    val_alb = augment_val_data(val_data_)
    
    #train_final, val_final = optimize_data(train_alb, val_alb)
    
    train_alb_mod = train_alb.map(onehot, num_parallel_calls=AUTOTUNE)
    train_final = train_alb_mod.prefetch(buffer_size=AUTOTUNE)
    
    val_alb_mod = val_alb.map(onehot, num_parallel_calls=AUTOTUNE)
    val_final = val_alb_mod.prefetch(buffer_size=AUTOTUNE)

    
    model = create_model()
    print("Training fold no.: " + str(fold_number+1))

    model_name = "denoised effnetb4 "
    fold_name = "fold.h5"
    filepath = model_name + str(fold_number+1) + fold_name
    callbacks = [ModelCheckpoint(filepath=filepath, monitor='val_accuracy', save_best_only=True)]

    history = model.fit(train_final, steps_per_epoch=total_steps, epochs=epochs, validation_data=val_final, callbacks=callbacks)
    oof_accuracy.append(max(history.history["val_accuracy"]))
    fold_number += 1
    if fold_number == n_splits:
        print("Training finished!")

  opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt)

Training fold no.: 1
Epoch 1/20
107/107 [==============================] - 81s 445ms/step - loss: 1.3265 - accuracy: 0.5165 - val_loss: 0.9749 - val_accuracy: 0.6820
Epoch 2/20
107/107 [==============================] - 43s 403ms/step - loss: 0.7753 - accuracy: 0.7377 - val_loss: 0.7747 - val_accuracy: 0.7511
Epoch 3/20
107/107 [==============================] - 43s 399ms/step - loss: 0.7147 - accuracy: 0.7456 - val_loss: 0.6625 - val_accuracy: 0.7836
Epoch 4/20
107/107 [==============================] - 42s 391ms/step - loss: 0.6294 - accuracy: 0.8001 - val_loss: 0.5529 - val_accuracy: 0.8108
Epoch 5/20
107/107 [==============================] - 43s 406ms/step - loss: 0.5702 - accuracy: 0.8132 - val_loss: 0.5133 - val_accuracy: 0.8254
Epoch 6/20
107/107 [==============================] - 43s 406ms/step - loss: 0.5864 - accuracy: 0.8052 - val_loss: 0.4609 - val_accuracy: 0.8473
Epoch 7/20
107/107 [==================

Epoch 9/20
107/107 [==============================] - 45s 427ms/step - loss: 0.4595 - accuracy: 0.8330 - val_loss: 0.3703 - val_accuracy: 0.8889
Epoch 10/20
107/107 [==============================] - 42s 394ms/step - loss: 0.4714 - accuracy: 0.8380 - val_loss: 0.3683 - val_accuracy: 0.8884
Epoch 11/20
107/107 [==============================] - 42s 394ms/step - loss: 0.4853 - accuracy: 0.8412 - val_loss: 0.3658 - val_accuracy: 0.8882
Epoch 12/20
107/107 [==============================] - 42s 390ms/step - loss: 0.4494 - accuracy: 0.8463 - val_loss: 0.3645 - val_accuracy: 0.8884
Epoch 13/20
107/107 [==============================] - 42s 392ms/step - loss: 0.4683 - accuracy: 0.8480 - val_loss: 0.3635 - val_accuracy: 0.8882
Epoch 14/20
107/107 [==============================] - 41s 389ms/step - loss: 0.4435 - accuracy: 0.8540 - val_loss: 0.3651 - val_accuracy: 0.8872
Epoch 15/20
107/107 [==============================] - 42s 390ms/step - loss: 0.4858 - accuracy: 0.8310 - val_loss: 0.3661 - 

107/107 [==============================] - 37s 345ms/step - loss: 0.4862 - accuracy: 0.8369 - val_loss: 0.3777 - val_accuracy: 0.8812
Epoch 20/20
107/107 [==============================] - 37s 347ms/step - loss: 0.4401 - accuracy: 0.8645 - val_loss: 0.3801 - val_accuracy: 0.8805
Training finished!


In [43]:
print("Average Out-Of-Fold Accuracy: {:.2f}".format(np.mean(oof_accuracy)))

Average Out-Of-Fold Accuracy: 0.88
